In [ ]:
%matplotlib inline
%load_ext nengo.ipynb
import matplotlib.pyplot as plt
import seaborn as sns

prefix = '../plots/cc_'

# Communication Channel

This example demonstrates how to create a connections from one neuronal ensemble to another that behaves like a communication channel (that is, it transmits information without changing it). 

Network diagram:

      [Input] ---> (A) ---> (B)

An abstract input signal is fed into a first neuronal ensemble $A$, which then passes it on to another ensemble $B$. The result is that spiking activity in ensemble $B$ encodes the value from the Input.  

## Step 1: Create the Network

In [ ]:
import numpy as np
import nengo
nengo.cache.get_default_decoder_cache().invalidate()

from nengo.dists import Uniform
import nengo_detailed_neurons
from nengo_detailed_neurons.neurons import Bahr2, IntFire1
from nengo_detailed_neurons.synapses import ExpSyn, FixedCurrent

# Create a 'model' object to which we can add ensembles, connections, etc.  
model = nengo.Network(label="Communications Channel", seed=3145987)
with model:
    # Create an abstract input signal that oscillates as sin(t)
    sin = nengo.Node(lambda x: np.sin(x))
    
    # Create the neuronal ensembles
    num_A_neurons = 200
    num_B_neurons = 50
    A = nengo.Ensemble(num_A_neurons, dimensions=1, max_rates=Uniform(60, 80))
    B = nengo.Ensemble(num_B_neurons, dimensions=1, neuron_type=Bahr2(), max_rates=Uniform(60, 80))
    C = nengo.Ensemble(num_A_neurons, dimensions=1, max_rates=Uniform(60, 80))
    conn = nengo.Connection(B, C)
    
    # Connect the input to the first neuronal ensemble
    nengo.Connection(sin, A)
    
    # Connect the first neuronal ensemble to the second (this is the communication channel)
    solver = nengo.solvers.LstsqL2(True)
    nengo.Connection(A, B, solver=solver, synapse=ExpSyn(0.005))

## Step 2: Add Probes to Collect Data
Even this simple model involves many quantities that change over time, such as membrane potentials of individual neurons. Typically there are so many variables in a simulation that it is not practical to store them all. If we want to plot or analyze data from the simulation we have to "probe" the signals of interest. 

In [ ]:
with model:
    sin_probe = nengo.Probe(sin)
    A_probe = nengo.Probe(A, synapse=.01)  # ensemble output 
    B_probe = nengo.Probe(B, synapse=.01)
    C_probe = nengo.Probe(C, synapse=.01)
    A_spikes = nengo.Probe(A.neurons, 'spikes')
    B_spikes = nengo.Probe(B.neurons, 'spikes')
    B_input = nengo.Probe(B.neurons, 'input')
    voltage = nengo.Probe(B.neurons, 'voltage')
    current = nengo.Probe(B.neurons, 'current')

## Step 3: Run the Model!  

In [ ]:
sim = nengo.Simulator(model)

In [ ]:
n = Bahr2()
gain, bias = n.gain_bias([80], [-0.4])
n.rates([-0.2, 0., 0.2, 0.5, 1.], gain, bias)

In [ ]:
%time sim.run(np.pi)

In [ ]:
#for cells in nrnengo.builder.ens_to_cells.values():
#    for c in cells:
#        c.neuron.soma.gbar_nat = 0.5 * 226.616175
#        c.neuron.hillock.gbar_nat = 0.5 * 9512.289205
#        c.neuron.tuft.gbar_nat = 0.5 * 47.817841
#        c.neuron.iseg.gbar_nat = 0.5 * 13326.766938
#        c.neuron.recalculate_channel_densities()

In [ ]:
%time sim.run(np.pi)

In [ ]:
from nengo.utils.ensemble import tuning_curves
plt.plot(*tuning_curves(B, sim))
plt.xlabel("x")
plt.ylabel("Firing rate (1/s)")
plt.title("Approximate interpolated tuning curves")

In [ ]:
i = 0

plt.plot(sim.trange(), sim.data[current][:, i])
plt.xlabel("Time (s)")
plt.ylabel("Soma %d current" % i)
plt.show()

plt.plot(sim.trange(), sim.data[voltage][:, i])
plt.xlabel("Time (s)")
plt.ylabel("Soma %d voltage" % i)
plt.show()

## Step 4: Plot the Results

In [ ]:
plt.figure(figsize=(9, 3))
plt.subplot(1, 3, 1)
plt.title("Input")
plt.plot(sim.trange(), sim.data[sin_probe])
plt.xlabel("t")
plt.xlim(0, 2 * np.pi)
plt.ylim(-1.2, 1.2)
plt.subplot(1, 3, 2)
plt.title("A ({} standard LIF)".format(num_A_neurons))
plt.plot(sim.trange(), sim.data[A_probe])
plt.xlabel("t")
plt.gca().set_yticklabels([])
plt.xlim(0, 2 * np.pi)
plt.ylim(-1.2, 1.2)
plt.subplot(1,3,3)
plt.title("B ({} compartmental)".format(num_B_neurons))
plt.plot(sim.trange(), sim.data[B_probe])
plt.xlabel("t")
plt.gca().set_yticklabels([])
plt.xlim(0, 2 * np.pi)
plt.ylim(-1.2, 1.2)

In [ ]:
dt = 1000.0
sns.set_style('white')
plt.figure(figsize=(18, 4))
plt.eventplot([np.where(x)[0] / dt for x in sim.data[A_spikes].T[:50, :] if np.any(x)], colors=[(0, 0, 0, 1)], linewidth=1)
plt.title("Spike raster of A population (first 50 neurons)")
plt.xlabel("t")
plt.ylabel("Neuron index")
plt.xlim(0, 2 * np.pi)
plt.ylim(-0.5, 49.5)

In [ ]:
dt = 1000.0
sns.set_style('white')
plt.figure(figsize=(18, 4))
spikes = [np.where(x)[0] / dt for x in sim.data[B_spikes].T if np.any(x)]
plt.eventplot(spikes, colors=[(0, 0, 0, 1)], linewidth=1)
plt.title("Spike raster of B population")
plt.xlabel("t")
plt.ylabel("Neuron index")
plt.xlim(0, 2 * np.pi)
plt.ylim(-0.5, len(spikes) - 0.5)

In [ ]:
sns.set_style('darkgrid')
plt.title("Voltage trace (soma)")
plt.xlabel("t (s)")
plt.ylabel("Membrane voltage (mV)")
plt.plot(sim.trange(), sim.data[voltage][:, 2])
plt.xlim(0, 2 * np.pi)

These plots show the idealized sinusoidal input, and estimates of the sinusoid that are decoded from the spiking activity of neurons in ensembles A and B. 

## Step 5: Using a Different Input Function
To drive the neural ensembles with different abstract inputs, it is convenient to use Python's "Lambda Functions". For example, try changing the `sin = nengo.Node` line to the following for higher-frequency input: 

    sin = nengo.Node(lambda t: np.sin(2*np.pi*t))